## 1. Install Required Dependencies

In [ ]:
!pip install -q fastapi uvicorn pyngrok xgboost scikit-learn pandas nest-asyncio

## 2. Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import pickle
import re
from urllib.parse import urlparse
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
from xgboost import XGBClassifier

import warnings
warnings.filterwarnings('ignore')

print("All libraries imported successfully!")

## 3. Load and Prepare Dataset

In [ ]:
# Load dataset
df = pd.read_csv('/content/sample_data/Phishing.csv')

# Rename target column
df.rename(columns={'CLASS_LABEL': 'label'}, inplace=True)

# Basic info
print(f"Dataset shape: {df.shape}")
print(f"\nLabel distribution:\n{df['label'].value_counts()}")
print(f"\nNull values: {df.isnull().sum().sum()}")
print(f"Duplicate rows: {df.duplicated().sum()}")

df.head()

## 4. Train-Test Split

In [ ]:
# Prepare features and target
X = df.drop(['id', 'label'], axis=1)
y = df['label']

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Training set: {X_train.shape}")
print(f"Test set: {X_test.shape}")
print(f"\nFeatures ({X_train.shape[1]}): {list(X_train.columns)}")

## 5. Train and Evaluate Multiple Models

In [ ]:
# Train Random Forest
print("Training Random Forest...")
rf_model = RandomForestClassifier(random_state=42, n_jobs=-1)
rf_model.fit(X_train, y_train)
rf_pred = rf_model.predict(X_test)

# Train Logistic Regression
print("Training Logistic Regression...")
lr_model = LogisticRegression(random_state=42, max_iter=1000, solver='liblinear')
lr_model.fit(X_train, y_train)
lr_pred = lr_model.predict(X_test)

# Train XGBoost
print("Training XGBoost...")
xgb_model = XGBClassifier(random_state=42, eval_metric='logloss')
xgb_model.fit(X_train, y_train)
xgb_pred = xgb_model.predict(X_test)

print("\nAll models trained successfully!")

In [ ]:
# Compare models
models_performance = {
    'Model': ['Random Forest', 'Logistic Regression', 'XGBoost'],
    'Accuracy': [
        accuracy_score(y_test, rf_pred),
        accuracy_score(y_test, lr_pred),
        accuracy_score(y_test, xgb_pred)
    ],
    'Precision': [
        precision_score(y_test, rf_pred),
        precision_score(y_test, lr_pred),
        precision_score(y_test, xgb_pred)
    ],
    'Recall': [
        recall_score(y_test, rf_pred),
        recall_score(y_test, lr_pred),
        recall_score(y_test, xgb_pred)
    ],
    'F1-Score': [
        f1_score(y_test, rf_pred),
        f1_score(y_test, lr_pred),
        f1_score(y_test, xgb_pred)
    ]
}

perf_df = pd.DataFrame(models_performance)
print("\n" + "="*60)
print("MODEL PERFORMANCE COMPARISON")
print("="*60)
print(perf_df.to_string(index=False))
print("="*60)

best_model_idx = perf_df['F1-Score'].idxmax()
print(f"\n🏆 Best Model: {perf_df.loc[best_model_idx, 'Model']} (F1-Score: {perf_df.loc[best_model_idx, 'F1-Score']:.4f})")

In [ ]:
# Visualize Model Comparison
import matplotlib.pyplot as plt
import seaborn as sns

# Set style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (14, 10)

# Create subplots
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('Model Performance Comparison', fontsize=16, fontweight='bold')

# 1. Bar chart for all metrics
ax1 = axes[0, 0]
metrics = ['Accuracy', 'Precision', 'Recall', 'F1-Score']
x = np.arange(len(perf_df['Model']))
width = 0.2

for i, metric in enumerate(metrics):
    ax1.bar(x + i*width, perf_df[metric], width, label=metric)

ax1.set_xlabel('Models', fontweight='bold')
ax1.set_ylabel('Score', fontweight='bold')
ax1.set_title('All Metrics Comparison')
ax1.set_xticks(x + width * 1.5)
ax1.set_xticklabels(perf_df['Model'])
ax1.legend()
ax1.set_ylim([0, 1.05])
ax1.grid(axis='y', alpha=0.3)

# 2. Grouped bar chart for better comparison
ax2 = axes[0, 1]
perf_df_melted = pd.melt(perf_df, id_vars=['Model'], var_name='Metric', value_name='Score')
models = perf_df['Model'].unique()
metric_names = ['Accuracy', 'Precision', 'Recall', 'F1-Score']
x2 = np.arange(len(metric_names))
width2 = 0.25

for i, model in enumerate(models):
    model_data = perf_df_melted[perf_df_melted['Model'] == model]
    scores = model_data['Score'].values
    ax2.bar(x2 + i*width2, scores, width2, label=model)

ax2.set_xlabel('Metrics', fontweight='bold')
ax2.set_ylabel('Score', fontweight='bold')
ax2.set_title('Metrics by Model')
ax2.set_xticks(x2 + width2)
ax2.set_xticklabels(metric_names)
ax2.legend()
ax2.set_ylim([0, 1.05])
ax2.grid(axis='y', alpha=0.3)

# 3. Heatmap
ax3 = axes[1, 0]
heatmap_data = perf_df.set_index('Model').T
sns.heatmap(heatmap_data, annot=True, fmt='.4f', cmap='YlGnBu', ax=ax3, 
            cbar_kws={'label': 'Score'}, linewidths=0.5)
ax3.set_title('Performance Heatmap')
ax3.set_xlabel('')
ax3.set_ylabel('Metrics', fontweight='bold')

# 4. Radar chart
ax4 = axes[1, 1]
categories = ['Accuracy', 'Precision', 'Recall', 'F1-Score']
angles = np.linspace(0, 2 * np.pi, len(categories), endpoint=False).tolist()
angles += angles[:1]

ax4 = plt.subplot(2, 2, 4, projection='polar')
for idx, model in enumerate(perf_df['Model']):
    values = perf_df.iloc[idx][['Accuracy', 'Precision', 'Recall', 'F1-Score']].tolist()
    values += values[:1]
    ax4.plot(angles, values, 'o-', linewidth=2, label=model)
    ax4.fill(angles, values, alpha=0.15)

ax4.set_xticks(angles[:-1])
ax4.set_xticklabels(categories)
ax4.set_ylim(0, 1)
ax4.set_title('Radar Chart Comparison', pad=20)
ax4.legend(loc='upper right', bbox_to_anchor=(1.3, 1.1))
ax4.grid(True)

plt.tight_layout()
plt.show()

# Print summary statistics
print("\n" + "="*70)
print("SUMMARY STATISTICS")
print("="*70)
print(f"Highest Accuracy:  {perf_df.loc[perf_df['Accuracy'].idxmax(), 'Model']} ({perf_df['Accuracy'].max():.4f})")
print(f"Highest Precision: {perf_df.loc[perf_df['Precision'].idxmax(), 'Model']} ({perf_df['Precision'].max():.4f})")
print(f"Highest Recall:    {perf_df.loc[perf_df['Recall'].idxmax(), 'Model']} ({perf_df['Recall'].max():.4f})")
print(f"Highest F1-Score:  {perf_df.loc[perf_df['F1-Score'].idxmax(), 'Model']} ({perf_df['F1-Score'].max():.4f})")
print("="*70)

## 6. Hyperparameter Tuning for XGBoost

In [ ]:
# Hyperparameter tuning
print("Performing hyperparameter tuning on XGBoost...")

param_grid = {
    'n_estimators': [100, 200],
    'learning_rate': [0.01, 0.1],
    'max_depth': [3, 5, 7],
    'subsample': [0.8, 0.9]
}

grid_search = GridSearchCV(
    estimator=XGBClassifier(random_state=42, eval_metric='logloss'),
    param_grid=param_grid,
    cv=3,
    scoring='f1',
    n_jobs=-1,
    verbose=1
)

grid_search.fit(X_train, y_train)

print(f"\nBest parameters: {grid_search.best_params_}")
print(f"Best F1-score: {grid_search.best_score_:.4f}")

# Get tuned model
tuned_model = grid_search.best_estimator_
tuned_pred = tuned_model.predict(X_test)

print(f"\nTuned Model Performance:")
print(f"Accuracy:  {accuracy_score(y_test, tuned_pred):.4f}")
print(f"Precision: {precision_score(y_test, tuned_pred):.4f}")
print(f"Recall:    {recall_score(y_test, tuned_pred):.4f}")
print(f"F1-Score:  {f1_score(y_test, tuned_pred):.4f}")

## 7. Save the Best Model

In [ ]:
# Save the tuned model
model_filename = 'phishing_detector_model.pkl'

with open(model_filename, 'wb') as f:
    pickle.dump(tuned_model, f)

print(f"✅ Model saved as '{model_filename}'")

# Verify the save
with open(model_filename, 'rb') as f:
    loaded_model = pickle.load(f)
    
test_pred = loaded_model.predict(X_test[:5])
print(f"\n✅ Model loaded successfully and verified")
print(f"Sample predictions: {test_pred}")

## 8. Feature Extraction Function for URLs

In [ ]:
# Define sensitive keywords
SENSITIVE_WORDS = ['login', 'account', 'verify', 'update', 'bank', 'paypal', 'secure', 'signin', 'confirm']

def extract_features_from_url(url: str) -> pd.Series:
    """
    Extract 48 features from a URL string for phishing detection.
    
    Args:
        url: URL string to analyze
    
    Returns:
        pd.Series with 48 features aligned with training data
    """
    features = {}
    parsed_url = urlparse(url)
    
    scheme = parsed_url.scheme.lower() if parsed_url.scheme else ''
    hostname = parsed_url.hostname if parsed_url.hostname else ''
    path = parsed_url.path
    query = parsed_url.query
    
    # URL-based features
    features['NumDots'] = float(url.count('.'))
    features['SubdomainLevel'] = float(hostname.count('.'))
    features['PathLevel'] = float(path.count('/'))
    features['UrlLength'] = float(len(url))
    features['NumDash'] = float(url.count('-'))
    features['NumDashInHostname'] = float(hostname.count('-'))
    features['AtSymbol'] = float(1 if '@' in url else 0)
    features['TildeSymbol'] = float(1 if '~' in url else 0)
    features['NumUnderscore'] = float(url.count('_'))
    features['NumPercent'] = float(url.count('%'))
    features['NumQueryComponents'] = float(query.count('&') + (1 if query else 0))
    features['NumAmpersand'] = float(url.count('&'))
    features['NumHash'] = float(1 if '#' in url else 0)
    features['NumNumericChars'] = float(sum(c.isdigit() for c in url))
    features['NoHttps'] = float(1 if scheme == 'http' else 0)
    
    # IP address detection
    ip_pattern = r"^\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}$"
    features['IpAddress'] = float(1 if re.match(ip_pattern, hostname) else 0)
    
    # Domain features
    features['HttpsInHostname'] = float(1 if 'https' in hostname else 0)
    features['HostnameLength'] = float(len(hostname))
    features['PathLength'] = float(len(path))
    features['QueryLength'] = float(len(query))
    features['DoubleSlashInPath'] = float(1 if '//' in path and not path.startswith('//') else 0)
    features['NumSensitiveWords'] = float(sum(1 for word in SENSITIVE_WORDS if word in url.lower()))
    
    # Placeholder features (not computable without HTML content)
    features['RandomString'] = 0.0
    features['DomainInSubdomains'] = 0.0
    features['DomainInPaths'] = 0.0
    features['EmbeddedBrandName'] = 0.0
    features['PctExtHyperlinks'] = 0.0
    features['PctExtResourceUrls'] = 0.0
    features['ExtFavicon'] = 0.0
    features['InsecureForms'] = 0.0
    features['RelativeFormAction'] = 0.0
    features['ExtFormAction'] = 0.0
    features['AbnormalFormAction'] = 0.0
    features['PctNullSelfRedirectHyperlinks'] = 0.0
    features['FrequentDomainNameMismatch'] = 0.0
    features['FakeLinkInStatusBar'] = 0.0
    features['RightClickDisabled'] = 0.0
    features['PopUpWindow'] = 0.0
    features['SubmitInfoToEmail'] = 0.0
    features['IframeOrFrame'] = 0.0
    features['MissingTitle'] = 0.0
    features['ImagesOnlyInForm'] = 0.0
    features['SubdomainLevelRT'] = 1.0
    features['UrlLengthRT'] = 0.0
    features['PctExtResourceUrlsRT'] = 1.0
    features['AbnormalExtFormActionR'] = 1.0
    features['ExtMetaScriptLinkRT'] = -1.0
    features['PctExtNullSelfRedirectHyperlinksRT'] = 1.0
    
    # Return as Series aligned with training features
    feature_series = pd.Series(features)
    return feature_series.reindex(X_train.columns, fill_value=0.0)

# Test the function
test_url = "https://www.google.com/search?q=test"
test_features = extract_features_from_url(test_url)
print(f"✅ Feature extraction function ready")
print(f"Extracted {len(test_features)} features from URL")
print(f"Sample: NumDots={test_features['NumDots']}, UrlLength={test_features['UrlLength']}")

## 9. Setup FastAPI Application with ngrok

In [ ]:
import sys
import time
import requests
import json
import socket
import uvicorn
import nest_asyncio
from pyngrok import ngrok
from fastapi import FastAPI, HTTPException
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
from threading import Thread

# Allow nested event loops (required for Colab)
nest_asyncio.apply()

# Initialize FastAPI app
app = FastAPI(title="Phishing Detection API", version="1.0")

# Enable CORS for frontend integration
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # In production, specify your frontend domain
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Request model
class URLInput(BaseModel):
    url: str

# Response model
class PredictionResponse(BaseModel):
    url: str
    prediction: int
    result: str
    confidence: str

# Load the trained model
with open('phishing_detector_model.pkl', 'rb') as f:
    model = pickle.load(f)

@app.get("/")
async def root():
    return {
        "message": "Phishing Detection API",
        "status": "active",
        "endpoints": {
            "/predict": "POST - Predict if a URL is phishing",
            "/health": "GET - Health check"
        }
    }

@app.get("/health")
async def health():
    return {"status": "healthy", "model_loaded": model is not None}

@app.post("/predict", response_model=PredictionResponse)
async def predict_phishing(url_input: URLInput):
    """
    Predict if a URL is phishing or legitimate.
    
    Returns:
    - prediction: 0 (legitimate) or 1 (phishing)
    - result: 'Legitimate' or 'Phishing'
    - confidence: Model confidence level
    """
    try:
        # Extract features from URL
        features = extract_features_from_url(url_input.url)
        
        # Convert to DataFrame for prediction
        input_df = pd.DataFrame([features])
        
        # Make prediction
        prediction = int(model.predict(input_df)[0])
        
        # Get probability for confidence
        proba = model.predict_proba(input_df)[0]
        confidence = f"{max(proba) * 100:.2f}%"
        
        result = "Phishing" if prediction == 1 else "Legitimate"
        
        return {
            "url": url_input.url,
            "prediction": prediction,
            "result": result,
            "confidence": confidence
        }
    
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Prediction failed: {str(e)}")

print("✅ FastAPI application configured successfully!")

## 10. Start the API Server with ngrok Tunnel

In [ ]:
# IMPORTANT: Set your ngrok auth token here
# Get your token from: https://dashboard.ngrok.com/get-started/your-authtoken
NGROK_AUTH_TOKEN = "YOUR_NGROK_AUTH_TOKEN_HERE"  # Replace with your token

# Kill any existing ngrok tunnels
ngrok.kill()

# Set ngrok auth token
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# Start uvicorn server in a separate thread
def run_server():
    uvicorn.run(app, host="0.0.0.0", port=8000, log_level="error")

server_thread = Thread(target=run_server, daemon=True)
server_thread.start()

# Wait for server to start
time.sleep(3)

# Create ngrok tunnel
public_url = ngrok.connect(8000).public_url

print("\n" + "="*70)
print("🚀 API SERVER IS RUNNING!")
print("="*70)
print(f"\n📡 Public URL: {public_url}")
print(f"\n🔗 API Endpoints:")
print(f"   - Root:    {public_url}/")
print(f"   - Health:  {public_url}/health")
print(f"   - Predict: {public_url}/predict (POST)")
print(f"\n📝 API Documentation: {public_url}/docs")
print("\n" + "="*70)
print("\n💡 Use this URL in your frontend to make predictions!")
print("\n⚠️  Keep this cell running to maintain the server")
print("="*70)

## 11. Test the API

In [ ]:
# Test the API with sample URLs
import requests
import json

API_URL = public_url + "/predict"
headers = {'Content-Type': 'application/json'}

# Test URLs
test_urls = [
    "https://www.google.com",
    "http://paypal-secure.verify-account.com/login",
    "https://github.com/microsoft/vscode",
    "http://192.168.1.1/admin/verify?user=admin"
]

print("\n" + "="*70)
print("🧪 TESTING API")
print("="*70 + "\n")

for test_url in test_urls:
    try:
        response = requests.post(
            API_URL,
            headers=headers,
            data=json.dumps({"url": test_url}),
            timeout=10
        )
        
        if response.status_code == 200:
            result = response.json()
            status_icon = "🚨" if result['prediction'] == 1 else "✅"
            print(f"{status_icon} URL: {test_url}")
            print(f"   Result: {result['result']} (Confidence: {result['confidence']})")
            print()
        else:
            print(f"❌ Error testing {test_url}: {response.status_code}")
            print()
    
    except Exception as e:
        print(f"❌ Exception testing {test_url}: {str(e)}")
        print()

print("="*70)

## 12. Frontend Integration Example

In [ ]:
print("""
╔════════════════════════════════════════════════════════════════╗
║              FRONTEND INTEGRATION GUIDE                        ║
╚════════════════════════════════════════════════════════════════╝

JavaScript/React Example:
─────────────────────────────────────────────────────────────────

const checkURL = async (url) => {
  try {
    const response = await fetch('""" + public_url + """/predict', {
      method: 'POST',
      headers: {
        'Content-Type': 'application/json',
      },
      body: JSON.stringify({ url: url })
    });
    
    const data = await response.json();
    console.log('Prediction:', data.result);
    console.log('Confidence:', data.confidence);
    return data;
  } catch (error) {
    console.error('Error:', error);
  }
};

// Usage
checkURL('https://example.com').then(result => {
  if (result.prediction === 1) {
    alert('⚠️ Warning: This URL appears to be PHISHING!');
  } else {
    alert('✅ This URL appears to be legitimate.');
  }
});

─────────────────────────────────────────────────────────────────

Python Example:
─────────────────────────────────────────────────────────────────

import requests

def check_url(url):
    response = requests.post(
        '""" + public_url + """/predict',
        json={'url': url}
    )
    return response.json()

result = check_url('https://example.com')
print(f"Result: {result['result']}")
print(f"Confidence: {result['confidence']}")

─────────────────────────────────────────────────────────────────

cURL Example:
─────────────────────────────────────────────────────────────────

curl -X POST """ + public_url + """/predict \\
  -H "Content-Type: application/json" \\
  -d '{"url": "https://example.com"}'

─────────────────────────────────────────────────────────────────

Response Format:
─────────────────────────────────────────────────────────────────
{
  "url": "https://example.com",
  "prediction": 0,              // 0 = Legitimate, 1 = Phishing
  "result": "Legitimate",       // Human-readable result
  "confidence": "98.75%"        // Model confidence
}

╚════════════════════════════════════════════════════════════════╝
""")

## Notes

### Important Points:

1. **Keep the server cell running** - The API will stop if you interrupt the cell
2. **ngrok URL changes** - Each time you restart, you'll get a new public URL
3. **Free tier limits** - ngrok free tier has connection limits
4. **CORS enabled** - The API accepts requests from any origin (adjust for production)
5. **Model Features** - The API extracts 48 features from each URL automatically

### Security Recommendations:

- In production, replace `allow_origins=["*"]` with your specific frontend domain
- Add rate limiting to prevent abuse
- Implement authentication if needed
- Use HTTPS in production
- Consider deploying to a cloud platform for better reliability

### Model Performance:

- The tuned XGBoost model achieves ~98-99% accuracy
- Features include URL structure, domain characteristics, and suspicious patterns
- Some features require HTML content analysis (set to defaults in this version)

### Next Steps:

1. Integrate with your frontend application
2. Monitor API performance and errors
3. Collect feedback and retrain model periodically
4. Consider deploying to a production server (AWS, GCP, Azure, Heroku, etc.)